<a href="https://colab.research.google.com/github/2bar/2bar.github.io/blob/master/Document_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Doc2vecを用いて、文章の類似性を調べて行きます。

今回は、よく似ていると言われている、太宰治と芥川龍之介について、どちらの作品でも精神を病んだ人物を取り扱っている「人間失格」との「河童」の類似性について調べることとします。


今回は青空文庫のデータを読み込みます。

In [1]:
import zipfile
import os.path, urllib.request as request

In [2]:
URL1="https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip"
URL2="https://www.aozora.gr.jp/cards/000879/files/69_ruby_1321.zip"

次のコードは、プロジェクトフォルダに対象のファイルがなければダウンロードする内容です。 

まずは、人間失格をダウンロードします。

In [3]:
localfile1 = "301_ruby_5915.zip"

if not os.path.exists(localfile1):
    print("ファイルをダウンロードします")
    request.urlretrieve(URL1,localfile1)

ファイルをダウンロードします


In [4]:
import zipfile
zipfile = zipfile.ZipFile(localfile1, 'r')

In [5]:
with zipfile.open('ningen_shikkaku.txt', 'r') as file:
  bindata = file.read()
  ningen_original = bindata.decode('shift_jis')

ダウンロードした文章の全文になります。

In [6]:
print( ningen_original)

人間失格
太宰治

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）従姉妹《いとこ》

｜：ルビの付く文字列の始まりを特定する記号
（例）昔｜気質《かたぎ》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
（例）［＃３字下げ］はしがき［＃「はしがき」は大見出し］
-------------------------------------------------------

［＃３字下げ］はしがき［＃「はしがき」は大見出し］


　私は、その男の写真を三葉、見たことがある。
　一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？　けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、
「可愛い坊ちゃんですね」
　といい加減なお世辞を言っても、まんざら空《から》お世辞に聞えないくらいの、謂《い》わば通俗の「可愛らしさ」みたいな影もその子供の笑顔に無いわけではないのだが、しかし、いささかでも、美醜に就いての訓練を経て来たひとなら、ひとめ見てすぐ、
「なんて、いやな子供だ」
　と頗《すこぶ》る不快そうに呟《つぶや》き、毛虫でも払いのける時のような手つきで、その写真をほうり投げるかも知れない。
　まったく、その子供の笑顔は、よく見れば見るほど、何とも知れず、イヤな薄気味悪いものが感ぜられて来る。どだい、それは、笑顔でない。この子は、少しも笑ってはいないのだ。その証拠には、この子は、両方のこぶしを固く握って立っている。人間は、こぶしを固く握りながら笑えるものでは無いのである。猿だ。猿の笑顔だ。ただ、顔に醜い皺《しわ》を寄せているだけなのである。「皺くちゃ坊ちゃん」とでも言いたくなるくらいの、まことに奇妙な、そうして、どこかけがらわしく、へんにひとをムカムカさせる表情の写真であった。私はこれまで、こんな不思議な表情の子供を見た事が、いちども

読み込んだ小説のデータから、正規表現を用いてルビなどを削除します。

In [7]:
import re
import pickle

ningen_original = re.split(r'\-{5,}',ningen_original)[2] # ヘッダーを取り除く
ningen_original = re.split(r'底本：',ningen_original)[0] # フッターを取り除く
ningen_original = ningen_original.strip()
print(ningen_original)

［＃３字下げ］はしがき［＃「はしがき」は大見出し］


　私は、その男の写真を三葉、見たことがある。
　一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？　けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、
「可愛い坊ちゃんですね」
　といい加減なお世辞を言っても、まんざら空《から》お世辞に聞えないくらいの、謂《い》わば通俗の「可愛らしさ」みたいな影もその子供の笑顔に無いわけではないのだが、しかし、いささかでも、美醜に就いての訓練を経て来たひとなら、ひとめ見てすぐ、
「なんて、いやな子供だ」
　と頗《すこぶ》る不快そうに呟《つぶや》き、毛虫でも払いのける時のような手つきで、その写真をほうり投げるかも知れない。
　まったく、その子供の笑顔は、よく見れば見るほど、何とも知れず、イヤな薄気味悪いものが感ぜられて来る。どだい、それは、笑顔でない。この子は、少しも笑ってはいないのだ。その証拠には、この子は、両方のこぶしを固く握って立っている。人間は、こぶしを固く握りながら笑えるものでは無いのである。猿だ。猿の笑顔だ。ただ、顔に醜い皺《しわ》を寄せているだけなのである。「皺くちゃ坊ちゃん」とでも言いたくなるくらいの、まことに奇妙な、そうして、どこかけがらわしく、へんにひとをムカムカさせる表情の写真であった。私はこれまで、こんな不思議な表情の子供を見た事が、いちども無かった。
　第二葉の写真の顔は、これはまた、びっくりするくらいひどく変貌《へんぼう》していた。学生の姿である。高等学校時代の写真か、大学時代の写真か、はっきりしないけれども、とにかく、おそろしく美貌の学生である。しかし、これもまた、不思議にも、生きている人間の感じはしなかった。学生服を着て、胸のポケットから白いハンケチを覗《のぞ》かせ、籐椅子《とういす》に腰かけて足を組み、そうして、やはり、笑っている。こんどの笑顔は、皺くちゃの猿の笑いでなく、かなり巧みな微笑になってはいるが、しかし、人間の笑いと、

In [8]:
ningen = re.sub('《[^》]+》', '', ningen_original) # ルビの削除
ningen = re.sub('［[^］]+］', '', ningen) # 読みの注意の削除
ningen = re.sub('[｜ 　「」\n]', '', ningen) # | と全角半角スペース、「」と改行の削除
ningen = re.sub(r'一\r\r|\r', '', ningen) # 先頭のゴミを削除

seperator = '。'  # 。をセパレータに指定
ningen_list = ningen.split(seperator)  # セパレーターを使って文章をリストに分割する
ningen_list.pop() # 最後の要素は空の文字列になるので、削除
ningen_list = [x+seperator for x in ningen_list]  # 文章の最後に。を追加
        
print(ningen_list)

with open('ningen_list.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(ningen_list, f)

['はしがき私は、その男の写真を三葉、見たことがある。', '一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹たちかと想像される）庭園の池のほとりに、荒い縞の袴をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。', '醜く？けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、可愛い坊ちゃんですねといい加減なお世辞を言っても、まんざら空お世辞に聞えないくらいの、謂わば通俗の可愛らしさみたいな影もその子供の笑顔に無いわけではないのだが、しかし、いささかでも、美醜に就いての訓練を経て来たひとなら、ひとめ見てすぐ、なんて、いやな子供だと頗る不快そうに呟き、毛虫でも払いのける時のような手つきで、その写真をほうり投げるかも知れない。', 'まったく、その子供の笑顔は、よく見れば見るほど、何とも知れず、イヤな薄気味悪いものが感ぜられて来る。', 'どだい、それは、笑顔でない。', 'この子は、少しも笑ってはいないのだ。', 'その証拠には、この子は、両方のこぶしを固く握って立っている。', '人間は、こぶしを固く握りながら笑えるものでは無いのである。', '猿だ。', '猿の笑顔だ。', 'ただ、顔に醜い皺を寄せているだけなのである。', '皺くちゃ坊ちゃんとでも言いたくなるくらいの、まことに奇妙な、そうして、どこかけがらわしく、へんにひとをムカムカさせる表情の写真であった。', '私はこれまで、こんな不思議な表情の子供を見た事が、いちども無かった。', '第二葉の写真の顔は、これはまた、びっくりするくらいひどく変貌していた。', '学生の姿である。', '高等学校時代の写真か、大学時代の写真か、はっきりしないけれども、とにかく、おそろしく美貌の学生である。', 'しかし、これもまた、不思議にも、生きている人間の感じはしなかった。', '学生服を着て、胸のポケットから白いハンケチを覗かせ、籐椅子に腰かけて足を組み、そうして、やはり、笑っている。', 'こんどの笑顔は、皺くちゃの猿の笑いでなく、かなり巧みな微笑になってはいるが、しかし、人間の笑いと、どこやら違う。', '血の重さ、とでも

形態素解析を用いて単語に分割します。
形態素解析ツールのjanomeを使いますが、最新版だと、動作が安定しないために、旧Verを使用します。

In [9]:
pip install janome==0.3.9

     |████████████████████████████████| 25.1MB 165kB/s 


人間失格について、単語に分ける分かち書き処理を行います。

In [10]:
from janome.tokenizer import Tokenizer
import pickle

t = Tokenizer()
w = []

for sentence in ningen_list:
  w.append(t.tokenize(sentence, wakati=True))
print(w)
with open('ningen_list.pickle', mode='wb') as f:
  pickle.dump(w, f)

[['はし', 'が', 'き', '私', 'は', '、', 'その', '男', 'の', '写真', 'を', '三', '葉', '、', '見', 'た', 'こと', 'が', 'ある', '。'], ['一葉', 'は', '、', 'その', '男', 'の', '、', '幼年', '時代', '、', 'と', 'でも', '言う', 'べき', 'で', 'あろ', 'う', 'か', '、', '十', '歳', '前後', 'か', 'と', '推定', 'さ', 'れる', '頃', 'の', '写真', 'で', 'あっ', 'て', '、', 'その', '子供', 'が', '大勢', 'の', '女', 'の', 'ひと', 'に', '取り', 'かこま', 'れ', '、', '（', 'それ', 'は', '、', 'その', '子供', 'の', '姉たち', '、', '妹', 'たち', '、', 'それから', '、', '従姉妹', 'たち', 'か', 'と', '想像', 'さ', 'れる', '）', '庭園', 'の', '池', 'の', 'ほとり', 'に', '、', '荒い', '縞', 'の', '袴', 'を', 'はい', 'て', '立ち', '、', '首', 'を', '三', '十', '度', 'ほど', '左', 'に', '傾け', '、', '醜く', '笑っ', 'て', 'いる', '写真', 'で', 'ある', '。'], ['醜く', '？', 'けれども', '、', '鈍い', '人', 'たち', '（', 'つまり', '、', '美醜', 'など', 'に', '関心', 'を', '持た', 'ぬ', '人', 'たち', '）', 'は', '、', '面白く', 'も', '何とも', '無い', 'よう', 'な', '顔', 'を', 'し', 'て', '、', '可愛い', '坊ちゃん', 'です', 'ね', 'と', 'いい加減', 'なお', '世辞', 'を', '言っ', 'て', 'も', '、', 'まんざら', '空', 'お', '世辞', 'に', '聞え', 'ない', 'くらい', 'の', '、', '謂わ', 'ば

保存ができているか、確認します。

In [11]:
with open('ningen_list.pickle', mode='rb') as f:
    ningen_list = pickle.load(f)

print(ningen_list)

[['はし', 'が', 'き', '私', 'は', '、', 'その', '男', 'の', '写真', 'を', '三', '葉', '、', '見', 'た', 'こと', 'が', 'ある', '。'], ['一葉', 'は', '、', 'その', '男', 'の', '、', '幼年', '時代', '、', 'と', 'でも', '言う', 'べき', 'で', 'あろ', 'う', 'か', '、', '十', '歳', '前後', 'か', 'と', '推定', 'さ', 'れる', '頃', 'の', '写真', 'で', 'あっ', 'て', '、', 'その', '子供', 'が', '大勢', 'の', '女', 'の', 'ひと', 'に', '取り', 'かこま', 'れ', '、', '（', 'それ', 'は', '、', 'その', '子供', 'の', '姉たち', '、', '妹', 'たち', '、', 'それから', '、', '従姉妹', 'たち', 'か', 'と', '想像', 'さ', 'れる', '）', '庭園', 'の', '池', 'の', 'ほとり', 'に', '、', '荒い', '縞', 'の', '袴', 'を', 'はい', 'て', '立ち', '、', '首', 'を', '三', '十', '度', 'ほど', '左', 'に', '傾け', '、', '醜く', '笑っ', 'て', 'いる', '写真', 'で', 'ある', '。'], ['醜く', '？', 'けれども', '、', '鈍い', '人', 'たち', '（', 'つまり', '、', '美醜', 'など', 'に', '関心', 'を', '持た', 'ぬ', '人', 'たち', '）', 'は', '、', '面白く', 'も', '何とも', '無い', 'よう', 'な', '顔', 'を', 'し', 'て', '、', '可愛い', '坊ちゃん', 'です', 'ね', 'と', 'いい加減', 'なお', '世辞', 'を', '言っ', 'て', 'も', '、', 'まんざら', '空', 'お', '世辞', 'に', '聞え', 'ない', 'くらい', 'の', '、', '謂わ', 'ば

河童についても同様の工程を行います。

In [12]:
localfile2 = "69_ruby_1321.zip"

if not os.path.exists(localfile2):
    print("ファイルをダウンロードします")
    request.urlretrieve(URL2,localfile2)

ファイルをダウンロードします


In [13]:
import zipfile
zipfile2 = zipfile.ZipFile(localfile2, 'r')

In [14]:
with zipfile2.open('kappa.txt', 'r') as file:
  bindata = file.read()
  kappa_original = bindata.decode('shift_jis')

In [15]:
print( kappa_original)

河童
どうか Kappa と発音してください。
芥川龍之介

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）両膝《りょうひざ》

｜：ルビの付く文字列の始まりを特定する記号
（例）三年｜前《まえ》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
（例）※［＃「言＋墟のつくり」、第4水準2-88-74］
-------------------------------------------------------

［＃８字下げ］序［＃「序」は中見出し］

　これはある精神病院の患者、――第二十三号がだれにでもしゃべる話である。彼はもう三十を越しているであろう。が、一見したところはいかにも若々しい狂人である。彼の半生の経験は、――いや、そんなことはどうでもよい。彼はただじっと両膝《りょうひざ》をかかえ、時々窓の外へ目をやりながら、（鉄格子《てつごうし》をはめた窓の外には枯れ葉さえ見えない樫《かし》の木が一本、雪曇りの空に枝を張っていた。）院長のＳ博士や僕を相手に長々とこの話をしゃべりつづけた。もっとも身ぶりはしなかったわけではない。彼はたとえば「驚いた」と言う時には急に顔をのけぞらせたりした。……
　僕はこういう彼の話をかなり正確に写したつもりである。もしまただれか僕の筆記に飽き足りない人があるとすれば、東京市外××村のＳ精神病院を尋ねてみるがよい。年よりも若い第二十三号はまず丁寧《ていねい》に頭を下げ、蒲団《ふとん》のない椅子《いす》を指さすであろう。それから憂鬱《ゆううつ》な微笑を浮かべ、静かにこの話を繰り返すであろう。最後に、――僕はこの話を終わった時の彼の顔色を覚えている。彼は最後に身を起こすが早いか、たちまち拳骨《げんこつ》をふりまわしながら、だれにでもこう怒鳴《どな》りつけるであろう。――「出て行け！　この悪党めが！　貴様も莫迦《ばか》な、嫉妬《しっと》深い、猥褻《わいせつ》な、ずうずうしい、うぬぼれきった、残酷な、虫のいい動物なんだろう。出ていけ！　この悪党めが！」

［＃８字下げ］一［＃「一」は中見出し］

　三年｜前《まえ》の夏のこ

In [16]:
import re
import pickle

kappa_original = re.split(r'\-{5,}',kappa_original)[2] 
kappa_original = re.split(r'底本：',kappa_original)[0] 
kappa_original = kappa_original.strip()
print(kappa_original)

［＃８字下げ］序［＃「序」は中見出し］

　これはある精神病院の患者、――第二十三号がだれにでもしゃべる話である。彼はもう三十を越しているであろう。が、一見したところはいかにも若々しい狂人である。彼の半生の経験は、――いや、そんなことはどうでもよい。彼はただじっと両膝《りょうひざ》をかかえ、時々窓の外へ目をやりながら、（鉄格子《てつごうし》をはめた窓の外には枯れ葉さえ見えない樫《かし》の木が一本、雪曇りの空に枝を張っていた。）院長のＳ博士や僕を相手に長々とこの話をしゃべりつづけた。もっとも身ぶりはしなかったわけではない。彼はたとえば「驚いた」と言う時には急に顔をのけぞらせたりした。……
　僕はこういう彼の話をかなり正確に写したつもりである。もしまただれか僕の筆記に飽き足りない人があるとすれば、東京市外××村のＳ精神病院を尋ねてみるがよい。年よりも若い第二十三号はまず丁寧《ていねい》に頭を下げ、蒲団《ふとん》のない椅子《いす》を指さすであろう。それから憂鬱《ゆううつ》な微笑を浮かべ、静かにこの話を繰り返すであろう。最後に、――僕はこの話を終わった時の彼の顔色を覚えている。彼は最後に身を起こすが早いか、たちまち拳骨《げんこつ》をふりまわしながら、だれにでもこう怒鳴《どな》りつけるであろう。――「出て行け！　この悪党めが！　貴様も莫迦《ばか》な、嫉妬《しっと》深い、猥褻《わいせつ》な、ずうずうしい、うぬぼれきった、残酷な、虫のいい動物なんだろう。出ていけ！　この悪党めが！」

［＃８字下げ］一［＃「一」は中見出し］

　三年｜前《まえ》の夏のことです。僕は人並みにリュック・サックを背負い、あの上高地《かみこうち》の温泉｜宿《やど》から穂高山《ほたかやま》へ登ろうとしました。穂高山へ登るのには御承知のとおり梓川《あずさがわ》をさかのぼるほかはありません。僕は前に穂高山はもちろん、槍《やり》ヶ｜岳《たけ》にも登っていましたから、朝霧の下《お》りた梓川の谷を案内者もつれずに登ってゆきました。朝霧の下りた梓川の谷を――しかしその霧はいつまでたっても晴れる景色《けしき》は見えません。のみならずかえって深くなるのです。僕は一時間ばかり歩いた後《のち》、一度は上高地の温泉宿へ引き返すことにしようかと思いました。けれども上高地へ引き返すにしても、とにかく霧の晴れるのを待った上にしなければ

In [17]:
kappa = re.sub('《[^》]+》', '', kappa_original) 
kappa = re.sub('［[^］]+］', '', kappa) 
kappa = re.sub('[｜ 　「」\n]', '', kappa) 
kappa = re.sub(r'一\r\r|\r', '', kappa) 

seperator = '。'  
kappa_list = kappa.split(seperator)  
kappa_list.pop() 
kappa_list = [x+seperator for x in kappa_list]  
        
print(kappa_list)

with open('kappa_list.pickle', mode='wb') as f:  
    pickle.dump(kappa_list, f)

['序これはある精神病院の患者、――第二十三号がだれにでもしゃべる話である。', '彼はもう三十を越しているであろう。', 'が、一見したところはいかにも若々しい狂人である。', '彼の半生の経験は、――いや、そんなことはどうでもよい。', '彼はただじっと両膝をかかえ、時々窓の外へ目をやりながら、（鉄格子をはめた窓の外には枯れ葉さえ見えない樫の木が一本、雪曇りの空に枝を張っていた。', '）院長のＳ博士や僕を相手に長々とこの話をしゃべりつづけた。', 'もっとも身ぶりはしなかったわけではない。', '彼はたとえば驚いたと言う時には急に顔をのけぞらせたりした。', '……僕はこういう彼の話をかなり正確に写したつもりである。', 'もしまただれか僕の筆記に飽き足りない人があるとすれば、東京市外××村のＳ精神病院を尋ねてみるがよい。', '年よりも若い第二十三号はまず丁寧に頭を下げ、蒲団のない椅子を指さすであろう。', 'それから憂鬱な微笑を浮かべ、静かにこの話を繰り返すであろう。', '最後に、――僕はこの話を終わった時の彼の顔色を覚えている。', '彼は最後に身を起こすが早いか、たちまち拳骨をふりまわしながら、だれにでもこう怒鳴りつけるであろう。', '――出て行け！この悪党めが！貴様も莫迦な、嫉妬深い、猥褻な、ずうずうしい、うぬぼれきった、残酷な、虫のいい動物なんだろう。', '出ていけ！この悪党めが！三年前の夏のことです。', '僕は人並みにリュック・サックを背負い、あの上高地の温泉宿から穂高山へ登ろうとしました。', '穂高山へ登るのには御承知のとおり梓川をさかのぼるほかはありません。', '僕は前に穂高山はもちろん、槍ヶ岳にも登っていましたから、朝霧の下りた梓川の谷を案内者もつれずに登ってゆきました。', '朝霧の下りた梓川の谷を――しかしその霧はいつまでたっても晴れる景色は見えません。', 'のみならずかえって深くなるのです。', '僕は一時間ばかり歩いた後、一度は上高地の温泉宿へ引き返すことにしようかと思いました。', 'けれども上高地へ引き返すにしても、とにかく霧の晴れるのを待った上にしなければなりません。', 'といって霧は一刻ごとにずんずん深くなるばかりなのです。', 'ええ、いっそ登ってしまえ。', '――僕はこう考えましたから、梓川の谷を離れな

In [18]:
from janome.tokenizer import Tokenizer
import pickle

t = Tokenizer()
w = []

for sentence in kappa_list:
  w.append(t.tokenize(sentence, wakati=True))
print(w)
with open('kappa_list.pickle', mode='wb') as f:
  pickle.dump(w, f)

[['序', 'これ', 'は', 'ある', '精神', '病院', 'の', '患者', '、', '――', '第', '二', '十', '三', '号', 'が', 'だれ', 'に', 'でも', 'しゃべる', '話', 'で', 'ある', '。'], ['彼', 'は', 'もう', '三', '十', 'を', '越し', 'て', 'いる', 'で', 'あろ', 'う', '。'], ['が', '、', '一見', 'し', 'た', 'ところ', 'は', 'いかにも', '若々しい', '狂人', 'で', 'ある', '。'], ['彼', 'の', '半生', 'の', '経験', 'は', '、', '――', 'いや', '、', 'そんな', 'こと', 'は', 'どう', 'で', 'も', 'よい', '。'], ['彼', 'は', 'ただ', 'じっと', '両', '膝', 'を', 'かかえ', '、', '時々', '窓', 'の', '外', 'へ', '目', 'を', 'やり', 'ながら', '、', '（', '鉄格子', 'を', 'はめ', 'た', '窓', 'の', '外', 'に', 'は', '枯れ葉', 'さえ', '見え', 'ない', '樫', 'の', '木', 'が', '一', '本', '、', '雪', '曇り', 'の', '空', 'に', '枝', 'を', '張っ', 'て', 'い', 'た', '。'], ['）', '院長', 'の', 'Ｓ', '博士', 'や', '僕', 'を', '相手', 'に', '長々', 'と', 'この', '話', 'を', 'しゃべり', 'つづけ', 'た', '。'], ['もっとも', '身ぶり', 'は', 'し', 'なかっ', 'た', 'わけ', 'で', 'は', 'ない', '。'], ['彼', 'は', 'たとえば', '驚い', 'た', 'と', '言う', '時', 'に', 'は', '急', 'に', '顔', 'を', 'のけぞら', 'せ', 'たり', 'し', 'た', '。'], ['…', '…', '僕', 'は', 'こういう', '彼', 'の', '話', 'を', '

In [19]:
with open('kappa_list.pickle', mode='rb') as f:
    kappa_list = pickle.load(f)

print(kappa_list)

[['序', 'これ', 'は', 'ある', '精神', '病院', 'の', '患者', '、', '――', '第', '二', '十', '三', '号', 'が', 'だれ', 'に', 'でも', 'しゃべる', '話', 'で', 'ある', '。'], ['彼', 'は', 'もう', '三', '十', 'を', '越し', 'て', 'いる', 'で', 'あろ', 'う', '。'], ['が', '、', '一見', 'し', 'た', 'ところ', 'は', 'いかにも', '若々しい', '狂人', 'で', 'ある', '。'], ['彼', 'の', '半生', 'の', '経験', 'は', '、', '――', 'いや', '、', 'そんな', 'こと', 'は', 'どう', 'で', 'も', 'よい', '。'], ['彼', 'は', 'ただ', 'じっと', '両', '膝', 'を', 'かかえ', '、', '時々', '窓', 'の', '外', 'へ', '目', 'を', 'やり', 'ながら', '、', '（', '鉄格子', 'を', 'はめ', 'た', '窓', 'の', '外', 'に', 'は', '枯れ葉', 'さえ', '見え', 'ない', '樫', 'の', '木', 'が', '一', '本', '、', '雪', '曇り', 'の', '空', 'に', '枝', 'を', '張っ', 'て', 'い', 'た', '。'], ['）', '院長', 'の', 'Ｓ', '博士', 'や', '僕', 'を', '相手', 'に', '長々', 'と', 'この', '話', 'を', 'しゃべり', 'つづけ', 'た', '。'], ['もっとも', '身ぶり', 'は', 'し', 'なかっ', 'た', 'わけ', 'で', 'は', 'ない', '。'], ['彼', 'は', 'たとえば', '驚い', 'た', 'と', '言う', '時', 'に', 'は', '急', 'に', '顔', 'を', 'のけぞら', 'せ', 'たり', 'し', 'た', '。'], ['…', '…', '僕', 'は', 'こういう', '彼', 'の', '話', 'を', '

リストの次元削減を行います。

In [20]:
import itertools

ningen_test = list(itertools.chain.from_iterable(ningen_list))
kappa_test = list(itertools.chain.from_iterable(kappa_list))
print(ningen_test)
print(kappa_test)

['はし', 'が', 'き', '私', 'は', '、', 'その', '男', 'の', '写真', 'を', '三', '葉', '、', '見', 'た', 'こと', 'が', 'ある', '。', '一葉', 'は', '、', 'その', '男', 'の', '、', '幼年', '時代', '、', 'と', 'でも', '言う', 'べき', 'で', 'あろ', 'う', 'か', '、', '十', '歳', '前後', 'か', 'と', '推定', 'さ', 'れる', '頃', 'の', '写真', 'で', 'あっ', 'て', '、', 'その', '子供', 'が', '大勢', 'の', '女', 'の', 'ひと', 'に', '取り', 'かこま', 'れ', '、', '（', 'それ', 'は', '、', 'その', '子供', 'の', '姉たち', '、', '妹', 'たち', '、', 'それから', '、', '従姉妹', 'たち', 'か', 'と', '想像', 'さ', 'れる', '）', '庭園', 'の', '池', 'の', 'ほとり', 'に', '、', '荒い', '縞', 'の', '袴', 'を', 'はい', 'て', '立ち', '、', '首', 'を', '三', '十', '度', 'ほど', '左', 'に', '傾け', '、', '醜く', '笑っ', 'て', 'いる', '写真', 'で', 'ある', '。', '醜く', '？', 'けれども', '、', '鈍い', '人', 'たち', '（', 'つまり', '、', '美醜', 'など', 'に', '関心', 'を', '持た', 'ぬ', '人', 'たち', '）', 'は', '、', '面白く', 'も', '何とも', '無い', 'よう', 'な', '顔', 'を', 'し', 'て', '、', '可愛い', '坊ちゃん', 'です', 'ね', 'と', 'いい加減', 'なお', '世辞', 'を', '言っ', 'て', 'も', '、', 'まんざら', '空', 'お', '世辞', 'に', '聞え', 'ない', 'くらい', 'の', '、', '謂わ', 'ば', '通

doc2vecを使って学習を行い、モデルを作成します。

文章のベクトル数値と類似度を表示します。

In [21]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

training_docs = []

sent1 = TaggedDocument(ningen_test,
                       tags=["ningen"])
sent2 = TaggedDocument(kappa_test,
                       tags=["kappa"])


training_docs.append(sent1)
training_docs.append(sent2)

# min_count：学習に使う単語の最低出現回数
# epochs:epochs数
# dm：学習モデル=DM
model = Doc2Vec(documents=training_docs,
                epochs=50,
                min_count=1,
                dm=1)
                
print(model.docvecs['ningen'])
print(model.docvecs.most_similar('ningen'))

[ 4.5701413  -1.2348936  -0.52464825  3.5840225  -1.0634255  -3.9132957
 -6.558838   -0.3509213  -0.59840125  3.0210345  -2.6339128  -1.0549399
 -6.342673   -2.2500956  -1.9981995  -0.43147177  3.186351   -5.0323763
 -1.3042531  -2.2460566  -1.7398101   7.319666    0.38939014  1.6355171
 -2.169472   -2.7632954   6.5066276   0.88700455 -2.5165086   1.0618044
  0.01444026  0.4459147  -5.016319    0.0807595  -1.0461515  -3.5205824
  0.64763    -1.5435598   1.0440439  -2.7221248  -2.9994771   4.198532
  0.8858028  -2.100692    1.8751438   0.5045555  -4.328773    2.0612023
  2.935754   -2.3919377  -5.4520154   0.41898963 -7.6985846  -2.4536967
 -4.2351437   3.5016406   3.5490541   2.631348   -7.3601356   1.1210752
 -2.6637323  -1.752613   -1.7039142   3.7066631   0.04345809  3.1222308
 -2.0679836   2.0638084  -2.8418589  -6.196074   -3.9707546   2.6765764
  8.400142   -0.09495918  7.7213655   3.0490358   2.6743886  -4.997518
  0.4417153   1.234837    5.6752815  -3.4606018   5.206531   -3.99

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


以上より人間失格と河童の類似度は約0.25と高くはないので、類似性があるとされているのは単語や助詞など以外のファクターが大きいと考えられる。

参考URL:

https://kento1109.hatenablog.com/entry/2017/11/15/181838

https://buildersbox.corp-sansan.com/entry/2019/04/10/110000#f-d99c2ee8

https://kitayamalab.wordpress.com/2016/11/14/python-%E3%81%A8-gensim-%E3%81%A7-doc2vec-%E3%82%92%E4%BD%BF%E3%81%86/

https://note.nkmk.me/python-list-flatten/